# ソルバーの実行

## 準備

In [7]:
import gurobi as gp
import datetime
import random
import numpy as np
random.seed(1)
import os

In [2]:
def group(i,l): # アイテムiがグループGに属しているかどうかをバイナリで返す関数
    if i in l:
        return 1
    else:
        return 0

In [18]:
def solve(moel_name,n,w,C,E,M,r,bin_cost):
    model = gp.Model(name = model_name)
    model.setParam('TimeLimit', 1000) #制限時間
    #解
    ans_x = {}
    ans_y = {}
    ans_b = {}
    #定数定義
    m = len(M)
    z = {}
    #変数定義
    x = {}
    y = {}
    b = {}
    for l in range(m):
        for k in range(n):
            y[k] = model.addVar(lb=0, ub=1, vtype='B', name='y[%s]'%k)
            b[k,l] = model.addVar(lb=0, ub=1, vtype='B', name='b[%s,%s]'%(k,l))
            for i in range(n):
                x[i,k] = model.addVar(lb=0, ub=1, vtype='B', name='x[%s,%s]'%(i,k))
                z[i,l] = group(i,M[l])
                #if l==0 and z[i,l]==1:
                    #print(i)
                
    #print(z)
    model.update()
        
    #目的関数
    ##model.setObjective(gp.quicksum(bin_cost*y[k] for k in range(n)) + gp.quicksum(r[l]*gp.quicksum(b[k,l] for k in range(n)) for l in range(m)) - sum(r), sense=gp.GRB.MINIMIZE)
    model.setObjective(gp.quicksum(y[k] for k in range(n)) + gp.quicksum(gp.quicksum(r[l]*b[k,l] for l in range(m)) for k in range(n)) - sum(r), sense=gp.GRB.MINIMIZE)

    #制約式
    for k in range(n):
        model.addConstr(gp.quicksum(w[i]*x[i,k] for i in range(n))<=C*y[k], 'Binlimit[%s]'%k) #ビンの容量を超えない
    for i in range(n):
        model.addConstr(gp.quicksum(x[i,k] for k in range(n))==1, 'Assign[%s]'%i) #すべてのアイテムはビンに詰め込まれている
        for k in range(n):
            model.addConstr(x[i,k]<=y[k], 'Strong[%s,%s]'%(i,k)) #強い定式化
    for e in E:
        for k in range(n):
            model.addConstr(x[e[0],k]+x[e[1],k]<=1,'Conflict[%s,%s]'%(e[0],e[1])) #コンフリクト
    for k in range(n):
        for l in range(m):
            for j in range(n):
                model.addConstr(b[k,l]>=x[j,k]*z[j,l],'Group[%s]'%l) #同じグループに属するアイテムは同じビンに割り当てられる
    
    #解の出力
    ##print("↓点線の間に、Gurobi Optimizerからログが出力")
    ##print("-" * 100)

    model.optimize()
    if model.Status == gp.GRB.INFEASIBLE:
        pass
    else:
        model.write(f"{model_name}.sol")
    
    ##print("-" * 100)
    ##print()
    
    if model.Status == gp.GRB.OPTIMAL:
        for v in model.getVars():
            if "x" in v.varName:
                #print("%s = %g"%(v.varName,v.x))
                ans_x[v.varName] = int(v.x)
            if "y" in v.varName:
                ans_y[v.varName] = int(v.x)
            if "b" in v.varName:
                ans_b[v.varName] = int(v.x)

        #print('Opt Value = ', model.ObjVal)
        #print("x",ans_x)
        #print("bin\t",sum(ans_y.values()))
        #print("penalty\t",sum(ans_b.values())*r[0])
        print("Objective value\t",sum(ans_y.values())+(sum(ans_b.values()))*r[0]-sum(r))

        return model.ObjVal

In [19]:
#入力, 出力を書き出す階層を指定
path = os.path.abspath(".")
path

'/Users/okamoto/Desktop/yutaokamoto_mac/Seminar/benchmark'

In [20]:
for instance_name in ["w30"]:#,"w40","w50","w60"]:
    for instance_number in range(10):
        for density in range(10):
            
            if (instance_number==0) and (density==0):################################
                
                n = 0
                V = []
                C = 150
                w = []
                E = []
                M = []
                group_split_cost = 1
                r = []
                bin_cost = 5
                ans_x = []
                ans_y = []
                
                file_input = path + "/" + "input" + "/small/" + str("i"+str(instance_name[1:])) + "_"  + str("n"+str(instance_number)) + "_"  + str("d"+str(int(density*10))) + ".txt" #インスタンスの規模_インスタンスの番号_グラフの密度
                with open(file_input,mode="r") as f:
                    for line in f:
                        if "Number of items\t" in line:
                            n = int(line.split("\t")[-2])
                            V = list(range(n))
                        elif "Capacity of bins\t" in line:
                            C = int(line.split("\t")[-2])
                        elif "w\t" in line:
                            w.append(int(line.split("\t")[-1]))
                        elif "e\t" in line:
                            E.append(tuple(map(int,line.split("\t")[-1].rstrip("\n").strip("(").strip(")").split(","))))
                        elif "g\t" in line:
                            M.append(tuple(map(int,line.split("\t")[-1].rstrip("\n").strip("(").strip(")").strip(",").split(","))))
                        elif "Costs of spliting groups\t" in line:
                            r = [int(i) for i in line.split("\t")[-2][1:-1].split(",")]
                        elif "Costs of using bins\t" in line:
                            bin_cost = int(line.split("\t")[-2])
                            
                #print(M[0])
                
                #コンフリクト
                conf={}
                n_conf = len(E)
                for i in range(n_conf):
                    temp = [0]*n
                    temp[E[i][0]-1] = 1
                    temp[E[i][1]-1] = 1
                    conf[f"conf{i}"] = temp

                file_output =  path + "/" + "output" + "/" + "Gurobi" + "/small/" + str("i"+str(instance_name[1:])) + "_"  + str("n"+str(instance_number)) + "_"  + str("d"+str(int(density*10))) + "_" + "1000s"
                model_name = str(file_output)

                objval = 0
                r = list(map(lambda x:x/bin_cost,r))
                bin_cost = bin_cost/bin_cost
                objval = solve(model_name,n,w,C,E,M,r,bin_cost)
                
                """file_output =  path + "/" + "output" + "/" + "Gurobi" + "/small/" + str("i"+str(instance_name[1:])) + "_"  + str("n"+str(instance_number)) + "_"  + str("d"+str(int(density*10))) + "_" + "1000s" + ".txt"
                with open(file_output,mode="w") as f:
                    f.write("Object value\t"+str(objval)+"\n\n")"""

Changed value of parameter TimeLimit to 1000.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 4560 rows, 3840 columns and 8130 nonzeros
Variable types: 0 continuous, 3840 integer (3840 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [2e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 25.2000000
Presolve removed 2970 rows and 2790 columns
Presolve time: 0.03s
Presolved: 1590 rows, 1050 columns, 6450 nonzeros
Variable types: 0 continuous, 1050 integer (1050 binary)

Root relaxation: objective 1.210667e+01, 2578 iterations, 0.08 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   12.10667    0   75   25.20000   12.10667  52.0%     -    0s
H    0     0                      15.8000000   12.10667  23.4%     -    0s
H    0     0           

# 解の図示 

## ビンごとに辞書に登録 

In [21]:
node_dict = {}
for Item,Bin in ans_x:
    if ans_x[Item,Bin] == 1:
        Item = int(Item)
        Bin = int(Bin)
        try:
            node_dict[Bin] = node_dict[Bin] + [Item]
        except:
            node_dict[Bin] = [Item]
        
node_dict

{0: [0]}

In [16]:
node_dict = {}
for Item,Bin in ans_x:
    if ans_x[Item,Bin] == 1:
        Item = int(Item)
        Bin = int(Bin)
        node_dict.setdefault(Bin,[])
        node_dict[Bin] = node_dict[Bin] + [Item]
    
node_dict

{0: [0]}

In [17]:
node_dict = {}
for Item,Bin in ans_x:
    if ans_x[Item,Bin] == 1:
        Item = int(Item)
        Bin = int(Bin)
        node_dict[Bin] = node_dict.setdefault(Bin,[]) + [Item]
    
node_dict

{0: [0]}

## 描画

In [18]:
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Agg")
import random
G = nx.Graph()

for v in range(n): # グラフにノードを追加
    G.add_node(v)
    
pos = nx.spring_layout(G, k=1)
plt.figure(figsize=(10,10))


for l in node_dict.values():
    node_color = random.sample(matplotlib.colors.cnames.keys(),1)
    nx.draw_networkx_nodes(G, pos, nodelist=[i for i in l], node_color=node_color, alpha=0.6, node_size=4000)
    nx.draw_networkx_labels(G, pos, font_size=50, font_family="Yu Gothic", font_weight="bold")
    
%matplotlib inline
plt.show()
#plt.savefig("figure.png")

In [19]:
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Agg")
import random
G = nx.Graph()

for v in range(n): # グラフにノードを追加
    G.add_node(v)
    
plt.figure(figsize=(10,10))

for b in range(n):
    node_color = random.sample(matplotlib.colors.cnames.keys(),1)
    for i in range(n):
        if ans_x[str(i),str(b)]==1:
            nx.draw_networkx_nodes(G, pos, nodelist=[i],node_color=node_color, alpha=0.6, node_size=4000)
            nx.draw_networkx_labels(G, pos, font_size=50, font_family="Yu Gothic", font_weight="bold")

plt.show()
#plt.savefig("figure.png")

KeyError: ('10', '0')

findfont: Font family ['Yu Gothic'] not found. Falling back to DejaVu Sans.


In [ ]:
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Agg")
import random
G = nx.Graph()

for v in range(n): # グラフにノードを追加
    G.add_node(v)
    
plt.figure(figsize=(10,10))

for b in map(lambda x:x[1],ans_x.keys()):
    node_color = random.sample(matplotlib.colors.cnames.keys(),1)
    for i in list(set(map(lambda x:x[0],ans_x.keys()))):
        if ans_x[str(i),str(b)]==1:
            nx.draw_networkx_nodes(G, pos, nodelist=[int(i)],node_color=node_color, alpha=0.6, node_size=4000)
            nx.draw_networkx_labels(G, pos, font_size=50, font_family="Yu Gothic", font_weight="bold")

plt.show()
#plt.savefig("figure.png")